# Set Up

In [0]:
!pip3 install numpy pandas gensim tqdm nltk pandas_ml imblearn tensorboardcolab
!pip3 install scikit-learn


In [0]:
import pandas as pd 
pd.options.mode.chained_assignment = None
import numpy as np 
from copy import deepcopy
from string import punctuation

import glob
import json

import gensim
from gensim.models.word2vec import Word2Vec 
TaggedDocument = gensim.models.doc2vec.TaggedDocument 

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

from nltk.tokenize import TweetTokenizer 
tokenizer = TweetTokenizer()

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from random import randint
from sklearn import preprocessing
from sklearn.preprocessing import scale

from sklearn.preprocessing import LabelBinarizer

from sklearn.utils import shuffle
import random
import pandas_ml as pdml

import tensorflow as tf

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score


# 1. Preprocessing Steps:


1.  Load Dataset (typhoon tweets, sentiment140)
2.  Build Word Embedding Model or Load Pre-trained Model
3.  Create Sentiment Features for Feature Extractor Model (LSTM)




##Helper functions for preprocessing

In [0]:
#loading sentiment140 training dataset,and project only (tweets text,class (0 or 1)). It also maps labels (4:1) in the original dataset.
def ingest():
    data = pd.read_csv('./Sentiment140/training.1600000.processed.noemoticon.csv', encoding = "ISO-8859-1") # use your path

    data.drop(['ItemID', 'SentimentSource'], axis=1, inplace=True)
    data = data[data.Sentiment.isnull() == False]
    data['Sentiment'] = data['Sentiment'].map( {4:1,0:0})
    data = data[data['SentimentText'].isnull() == False]
    data.reset_index(inplace=True)
    data.drop('index', axis=1, inplace=True)    
    data=shuffle(data) #randmoize sequence of data
    print(('dataset loaded with shape', data.shape))    

    return data

# extract tweets text and label; also it maps label (4) to (1) as a positive 
def postprocess(data, n=1600000): # loading 1.6 million tweets
    data = data.head(n)
    data['tokens'] = data['SentimentText'].progress_map(tokenize)  ## progress_map is a variant of the map function plus a progress bar. Handy to monitor DataFrame creations.
    data = data[data.tokens != 'NC']
    data.reset_index(inplace=True)
    data.drop('index', inplace=True, axis=1)
    return data

#tokenizing tweets: clean hashtags,usernames, and stop words. return list of words
def tokenize(tweet):
    try:
        tokens = tokenizer.tokenize(tweet.lower())
        tokens = list([t for t in tokens if not t.startswith('@')])
        tokens = list([t for t in tokens if not t.startswith('#')])
        tokens = list([t for t in tokens if not t.startswith('http')])
        tokens = list([t for t in tokens if not t.startswith('https')])
        
        #keep only text tweets, ignore numbers
        tokens=list([t for t in tokens if t.isalpha()])
        
        return tokens
    except:
        return 'NC'

# build a labeledSentence from tweet's text to train the word embedding model.
def labelizeTweets(tweets, label_type):
    labelized = []
    for i,v in tqdm(enumerate(tweets)):
        label = '%s_%s'%(label_type,i)
        labelized.append(TaggedDocument(v, [label]))
    return labelized

## 1.1 Load Sentiment140 Dataset

In [0]:
sentiment140=ingest() # loading 1.6 Million Labelled tweets
sentiment140=postprocess(sentiment140) # clearning and representing data as tweet and sentiment

In [0]:
# separate Training Features (X) from Labels (Y)

n=1600000 # data size 1.6 million tweets
x_sentiment,y_sentiment = np.array(sentiment140.head(n).tokens),np.array(sentiment140.head(n).Sentiment) 

## 1.1 Load Typhoon Tweets

In [0]:
typhoon_df=pd.DataFrame()
path ='./TED Dataset/Typhoons_tweets' # We provided tweets ids, please download full tweets data into Typhoons_tweets folder
allFiles = glob.glob(path + "/*.csv")

# merge all typhoon tweets into one file
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df) # append all tweets_lists into one list
    
typhoon_df = pd.concat(list_) # merage all tweets together.
typhoon_tweets=typhoon_df['text'].tolist()

## tokenize tweets as a list of words ## 
tweetsTokens=list()

#Tweets preprocessing
for tweet in tqdm(typhoon_tweets):
    tweet=str(tweet)
    tweetsTokens.append(tokenize(tweet)) # clean tweets and append to tweets_list

typhoon_tweets_tokens=tweetsTokens # tokenized tweets.

100%|██████████| 678982/678982 [00:59<00:00, 11422.47it/s]


## 2. Build a Word Embedding Model.
To train a word embedding model  -from scratch- RUN the first cell.


OR

Load a pre-trained word embedding model.


In [0]:
# ---------------------------- Build A Word Embedding Model (Word2vec) ----------------------- #

# combine all twees together. 
allTweets=sentiment140['tokens'].tolist()+typhoon_tweets_tokens

# building all word embedding (Words)
allTweets = labelizeTweets(allTweets, 'AllTWEETS')

n_dim=200 # word2vec dimension

word_emb = Word2Vec(size=n_dim, min_count=10,sg=1) #sg=1 Skipgram is used
word_emb.build_vocab([x.words for x in tqdm(allTweets)]) # words attribute by LabeledSentence
word_emb.train([x.words for x in tqdm(allTweets)],total_examples=len(allTweets),epochs=10)


In [0]:
# ----------------------- Load a Pre-trained Word Embedding Model -----------------#
word_emb=Word2Vec.load('./wordEmbedding.mdl')

# Train and Evaluate Baseline (RNN)

## Load Pre-processed Training and Test dataset

In [0]:
# n: number of batches in training dataset. 
# In our preprocessed data, each batch has 32 typhoon sample. Total batches is 79

def load_training_Batches(n=79): 
    typhoon_features=[]
    labels=[]
            
    for i in range(n):
        
        typhoon_features.append(np.load('./Training Data-Baseline/typhoon_features/typhoon_features'+str(i)+'.npy'))
        labels.append(np.int_(np.load('./Training Data-Baseline/labels/labels'+str(i)+'.npy')))

    return typhoon_features,labels
  
# In testing data, there is 19 batches of testing data. Each batch has 32 typhoon sample  
def load_testing_Batches(n=19): 
    typhoon_features=[]
    labels=[]

    for i in range(n):
        
        typhoon_features.append(np.load('./Testing Data-Baseline/typhoon_features/typhoon_features'+str(i)+'.npy'))
        labels.append(np.int_(np.load('./Testing Data-Baseline/labels/labels'+str(i)+'.npy')))
            
    return typhoon_features,labels
  

In [0]:
training_features,training_labels=load_training_Batches()
testing_features,testing_labels=load_testing_Batches()

## RNN Baseline Model

### RNN Model Architecture

In [0]:
typhoon_batchSize=32
featuresCount=12
typhoon_classes=4

learningrate=0.001
hidden_nodes=16

In [0]:
def LSTM_cell(hidden_layer_size, batch_size,number_of_layers, dropout=True, dropout_rate=0.6):
    
    layer = tf.contrib.rnn.BasicLSTMCell(hidden_layer_size)
        
    if dropout:
        layer = tf.contrib.rnn.DropoutWrapper(layer, output_keep_prob=dropout_rate)
        
    cell = tf.contrib.rnn.MultiRNNCell([layer]*number_of_layers)
    
    init_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, init_state


def output_layer(lstm_output, in_size, out_size):
    x = lstm_output[:, -1, :]
    weights = tf.Variable(tf.truncated_normal([in_size, out_size], stddev=0.05), name='output_layer_weights')
    bias = tf.Variable(tf.zeros([out_size]), name='output_layer_bias')
    
    output = tf.matmul(x, weights) + bias
    return output

In [0]:
hidden_layer_size=64
learning_rate=0.001
number_of_layers=1
dropout=True
dropout_rate=0.75
grad_clip_margin=4

def RNN_model(X_data): 
    
    X_data=tf.reshape(X_data,[typhoon_batchSize,featuresCount,1])
    
    cell, _ = LSTM_cell(hidden_layer_size, typhoon_batchSize, number_of_layers, dropout, dropout_rate)
    
    outputs, states = tf.nn.dynamic_rnn(cell, X_data, dtype=tf.float32)
    
    logits=output_layer(outputs, hidden_layer_size, typhoon_classes)
    
    return logits

## Computation Graph

In [0]:
tf.reset_default_graph()
RNN_graph=tf.Graph()

with RNN_graph.as_default():
    
    #prameters for Typhoon Model
    typhoon_X=tf.placeholder(name='Typhoon_Input',dtype=tf.float32,shape=[typhoon_batchSize,featuresCount])
    typhoon_ytrue=tf.placeholder(name='Typhoon_Label',dtype=tf.float32,shape=[typhoon_batchSize,typhoon_classes])
    
    with tf.name_scope("RNN_Model") as scope:                      
        
        typhoon_ypred=RNN_model(typhoon_X) # prediction with typhoon features only       
        
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            
            # Ensures that we execute the update_ops before performing the train_step
            rnn_loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                logits=typhoon_ypred,labels=typhoon_ytrue))
                                           
            y_p=tf.argmax(typhoon_ypred,1)

        
            #Cliping the gradient loss
            gradients = tf.gradients(rnn_loss, tf.trainable_variables())
            clipper_, _ = tf.clip_by_global_norm(gradients, grad_clip_margin)
            
            optimizer = tf.train.AdamOptimizer()
            rnn_opt = optimizer.apply_gradients(zip(gradients, tf.trainable_variables()))
            
            predict_ = tf.equal(tf.argmax(typhoon_ypred,1), tf.argmax(typhoon_ytrue,1))
            rnn_accuracy = tf.reduce_mean(tf.cast(predict_, tf.float32))
            
            y_p=tf.argmax(typhoon_ypred,1)

## Visualizing Model Training and Testing in Tensorboard

In [0]:
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback

tbc=TensorBoardColab()

## Execution Session

In [0]:
logdir1 = "./Graph/RNN_Model(training)/"
logdir2 = "./Graph/RNN_Model(testing)/"

epochs=100
training_iteration=79 
testing_iteration=19


with tf.Session(graph=RNN_graph) as sess:
    
    sess.run(tf.global_variables_initializer())   
    writer1 = tf.summary.FileWriter(logdir1, sess.graph)
    writer2 = tf.summary.FileWriter(logdir2, sess.graph)
    

    saver = tf.train.Saver()
        
    for j in range(epochs):
      
        accuracy=[]

        loss=[]
        
        for i in range(training_iteration):
                                    
            feed_dict={typhoon_X:training_features[i], typhoon_ytrue:training_labels[i]}
            
            sess.run([rnn_opt],feed_dict=feed_dict)                    
            loss_value,accuracy_value=sess.run([rnn_loss,rnn_accuracy],feed_dict=feed_dict)
            
                 
            
            accuracy.append(accuracy_value)
            loss.append(loss_value)
                
        value = tf.Summary.Value(tag="Loss",simple_value=np.mean(loss))
        summary = tf.Summary(value=[value])
        writer1.add_summary(summary,j)
            
        value = tf.Summary.Value(tag="Accuracy",simple_value=np.mean(accuracy))
        summary = tf.Summary(value=[value])
        writer1.add_summary(summary,j)

        # Evaluating testdata
        accuracy=[]
        precision=[]
        recall=[]
        f1=[]
        
        for i in range(testing_iteration):
                                    
            feed_dict={typhoon_X:testing_features[i], typhoon_ytrue:testing_labels[i]}                        
            
            loss_value,accuracy_value,typhoon_ypred_value=sess.run([rnn_loss,rnn_accuracy,y_p],feed_dict=feed_dict)
            
            accuracy.append(accuracy_value)
            loss.append(loss_value)
            
            
            #metrics
            y_true = np.argmax(testing_labels[i],1)
            
            accuracy.append(accuracy_value)
            precision.append(precision_score(y_true, typhoon_ypred_value,average='weighted'))
            recall.append(recall_score(y_true, typhoon_ypred_value,average='weighted'))
            f1.append(f1_score(y_true, typhoon_ypred_value,average='weighted'))
            
            
        
                                
        value = tf.Summary.Value(tag="Loss",simple_value=np.mean(loss))
        summary = tf.Summary(value=[value])
        writer2.add_summary(summary,j)
                    
        value = tf.Summary.Value(tag="Accuracy",simple_value=np.mean(accuracy))
        summary2 = tf.Summary(value=[value])
        writer2.add_summary(summary2,j)
      
      
      
      # ----- Printing Average Performance Results After Training 100 Epoches------------------------# 
    print ("Performance metric:",np.mean(accuracy),np.mean(precision),np.mean(recall),np.mean(f1))        
        
      
        


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Performance metric: 0.7763158 0.8172748766154161 0.7763157894736842 0.7671323709516361


# Training and Evaluate Joint-Model (LSTM+RNN)


## 1. Feature Extractor Model (LSTM)

In [0]:
lstm_size = 64
senti_batch_size = 48
learning_rate = 0.001
senti_classes=2 
embed_size=200


In [0]:

def LSTM_Model(senti_inputs,transfer_learning=False):
  
    data=tf.nn.embedding_lookup(wordVectors,senti_inputs) # tweets_model_wordVectors, word_embedding_vectors, gloveVectors, wordVectors
        
    lstmCell=tf.nn.rnn_cell.LSTMCell(num_units=lstm_size,name='basic_lstm_cell')
    lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.75)
    outputs, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

    weight = tf.Variable(tf.truncated_normal([lstm_size, senti_classes]))
    bias = tf.Variable(tf.constant(0.1, shape=[senti_classes]))
    
    value = tf.transpose(outputs, [1, 0, 2])
    last = tf.gather(value, int(value.get_shape()[0]) - 1)
    senti_logits = (tf.matmul(last, weight) + bias)
    
    senti_predictions=tf.nn.softmax(senti_logits)
    
    return senti_predictions

### 1.1 Loading Data for Feature Extractor Model

In [0]:
# Loading word embeddiing vocabs
words=list(word_emb.wv.vocab)
wordVectors=word_emb[word_emb.wv.vocab]

maxSeqLength = 20 # based on average count of words per tweets in training dataset

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [0]:
# Buidling words indices.
words_indices={} # from tweets2vec model

i=0
for w in words:
  words_indices[w]=i
  i+=1
  

In [0]:
sentiment_features = np.zeros((len(x_sentiment), maxSeqLength), dtype=int)

i=0 # instance counter
for instance in tqdm(x_sentiment):    
    vectors=np.zeros(maxSeqLength,dtype=int)
   
    j=0  # word counter
    for word in instance[:maxSeqLength]:
        if word in words:
            vectors[j]=words_indices[word] #get word index
            j+=1
    
    sentiment_features[i]=vectors
    i+=1

100%|██████████| 1600000/1600000 [13:56<00:00, 1913.06it/s]


### Encode label classes into one-hot vector

In [0]:
# representing class data as one-hot vectors
y_sentiment_ = np.array([y_sentiment]).reshape(-1)
one_hot_targets = np.eye(2)[y_sentiment_]

### 1.3 Split Data (train-test, 80%-20%)

In [0]:
# split data to train and test
split_frac = 0.8
split_idx = int(len(sentiment_features)*split_frac)

senti_train_x, senti_val_x = sentiment_features[:split_idx], sentiment_features[split_idx:]
senti_train_y, senti_val_y = one_hot_targets[:split_idx], one_hot_targets[split_idx:]

### 1.4 Load Batches of Sentiment140 dataset

In [0]:
def load_Sentiment_Training_Batch():
  
    labels = np.zeros([senti_batch_size,senti_classes], dtype=int)
    arr = np.zeros([senti_batch_size, maxSeqLength], dtype=int)
    
    for i in range(senti_batch_size):
        num = randint(1,len(senti_train_x)-1) # get random index
        
        labels[i]=senti_train_y[num]
        arr[i] = sentiment_features[num]
    return arr, labels

def load_Sentiment_Testing_Batch():
  
    labels = np.zeros([senti_batch_size,senti_classes], dtype=int)
    arr = np.zeros([senti_batch_size, maxSeqLength],dtype=int)
    
    for i in range(senti_batch_size):
        num = randint(1,len(senti_val_x)-1) # get random index
        
        labels[i]=senti_val_y[num]
        arr[i] = sentiment_features[num]
    
    return arr, labels

## 2. Joint Model (LSTM+RNN) - Load Training and Testing dataset

In [0]:
def load_training_Batches(n=79): 
    typhoon_features=[]
    labels=[]
    all_aggreTweets=[]
    tweetsCounts=[]
    
    for i in range(n):
        
        typhoon_features.append(np.load('./Training Data-Joint Model/typhoon_features/typhoon_features'+str(i)+'.npy'))
        labels.append(np.int_(np.load('./Training Data-Joint Model/labels/labels'+str(i)+'.npy')))
        
        all_aggreTweets.append(np.int_(np.load('./Training Data-Joint Model/all_aggreTweets/all_aggreTweets'+str(i)+'.npy')))
        tweetsCounts.append(np.int_(np.load('./Training Data-Joint Model/tweetsCounts/tweetsCounts'+str(i)+'.npy')))
    
    return typhoon_features,labels,all_aggreTweets,tweetsCounts
  
  
def load_testing_Batches(n=19): 
    typhoon_features=[]
    labels=[]
    all_aggreTweets=[]
    tweetsCounts=[]
    
    for i in range(n):
        
        typhoon_features.append(np.load('./Testing Data-Joint Model/typhoon_features/typhoon_features'+str(i)+'.npy'))
        labels.append(np.int_(np.load('./Testing Data-Joint Model/labels/labels'+str(i)+'.npy')))
        
        all_aggreTweets.append(np.int_(np.load('./Testing Data-Joint Model/all_aggreTweets/all_aggreTweets'+str(i)+'.npy')))
        tweetsCounts.append(np.int_(np.load('./Testing Data-Joint Model/tweetsCounts/tweetsCounts'+str(i)+'.npy')))
    
    return typhoon_features,labels,all_aggreTweets,tweetsCounts
  


In [0]:
training_features,training_labels,Tweets_train,tweetsCounts_train=load_training_Batches()
testing_features,testing_labels,Tweets_test,tweetsCounts_test=load_testing_Batches()

## Visualize Training and Testing in Tensorboard

In [0]:
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback

tbc=TensorBoardColab()

## 3. Joint-Model (LSTM+RNN) - Computation Graph

In [0]:
tf.reset_default_graph()
LSTM_RNN_Graph=tf.Graph()

with LSTM_RNN_Graph.as_default():
    
    with tf.name_scope('Tweets_Placeholders') as scope:
        tweets_inputs=tf.placeholder(tf.int32,[None,maxSeqLength],name='AllTweets_input')
        tweets_labels=tf.placeholder(tf.int32,[senti_batch_size,senti_classes],name='sentiment_labels')
        tweets_Sizes=tf.placeholder(tf.int32,[typhoon_batchSize],name='AggreTweets_Sizes')
    
    with tf.name_scope('Typhoon_Placeholder') as scope:          
        typhoon_X=tf.placeholder(name='Typhoon_Input',dtype=tf.float32,shape=[None,None])
        typhoon_ytrue=tf.placeholder(name='Typhoon_Label',dtype=tf.float32,shape=[None,typhoon_classes])
        
    with tf.device('/gpu:0'):
        with tf.name_scope('LSTM_Model') as scope:
            
            tweets_pred=LSTM_Model(tweets_inputs,transfer_learning=False)
            
            sentiTweets_pred=tweets_pred[:senti_batch_size] 
            typhoonTweet_pred=tweets_pred[senti_batch_size:]

            lstm_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=sentiTweets_pred,labels=tweets_labels)) 

    with tf.name_scope('Tweets_Statistics') as scope:
            # Tweets aggrgation and Analysis
            def condition(tweets_Sizes,typhoonTweet_pred,typhoon_x,newTyphoon_x,j,i):            
                return tf.less(i, tf.shape(tweets_Sizes)[0])

            def body(tweets_Sizes,typhoonTweet_pred,typhoon_x,newTyphoon_x,j,i):

                # get prediction of one typhoon at a time; here we do slicing to typhoonTweet_pred tensor
                # i: index of aggreTweet size in Tweets_size tensor; while j step-by index in predictions tensor
                typhoon_prediction=typhoonTweet_pred[j:tweets_Sizes[i]] 

                tweetsCount=tf.cast(tf.nn.embedding_lookup(tweets_Sizes,i),tf.float32)

                # get mean and variance
                _, variance = tf.nn.moments(typhoonTweet_pred, [0])    
                std=tf.sqrt(variance)
                tweets_features=tf.concat([[tweetsCount],std],0)
                tweets_features_Normalized=tf.nn.l2_normalize(tweets_features,dim=0)
                tweets_features_Normalized=tweets_features

                # Get Current typhoon instance from Typhoon batch:
                current_Typhoon=tf.nn.embedding_lookup(typhoon_x,i)
                # extend tweets feature with typhoon     
                new_Typhoon=tf.concat([current_Typhoon,tweets_features_Normalized],0)
                newTyphoon_x=newTyphoon_x.write(i,new_Typhoon)

                return tweets_Sizes,typhoonTweet_pred,typhoon_x,newTyphoon_x,tweets_Sizes[i],i+1

            newTyphoon_x= tf.TensorArray(dtype=tf.float32,size=0,dynamic_size=True,element_shape=(featuresCount,))    
            _,_,_,newTyphoon_x,_,_=tf.while_loop(condition, body, [tweets_Sizes,typhoonTweet_pred,typhoon_X,newTyphoon_x, 0,0])       
            newTyphoon_x = newTyphoon_x.stack()
            
    with tf.device('/gpu:0'):
        with tf.name_scope('RNN_Model') as scope:
          
          update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
          
          with tf.control_dependencies(update_ops): 
            
            typhoon_ypred=RNN_model(newTyphoon_x)                          

            # Ensures that we execute the update_ops before performing the train_step            
            rnn_loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=typhoon_ypred,labels=typhoon_ytrue))                       
              # joint training loss function              
            joint_Loss=lstm_loss+rnn_loss  
                        
                        
            #Cliping the gradient loss
            gradients = tf.gradients(joint_Loss, tf.trainable_variables())
            clipper_, _ = tf.clip_by_global_norm(gradients, grad_clip_margin)
            
            optimizer = tf.train.AdamOptimizer()
            rnn_opt = optimizer.apply_gradients(zip(gradients, tf.trainable_variables()))
            
            predict_opt=tf.equal(tf.argmax(typhoon_ypred,1), tf.argmax(typhoon_ytrue,1))
            joint_accuracy = tf.reduce_mean(tf.cast(predict_opt, tf.float32))
            
            y_p=tf.argmax(typhoon_ypred,1)


### Joint Model (LSTM+RNN) - Execution Session

In [0]:
logdir1 = "./Graph/LSTM-RNN(training)"
logdir2 = "./Graph/LSTM-RNN(test)"

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
config.allow_soft_placement=True
config.log_device_placement=True

epochs=100
training_iteration=79 
testing_iteration=19


with tf.Session(graph=LSTM_RNN_Graph,config=config) as sess:
    sess.run(tf.global_variables_initializer())   
    
    writer1 = tf.summary.FileWriter(logdir1, sess.graph)
    writer2 = tf.summary.FileWriter(logdir2, sess.graph)
    
    saver = tf.train.Saver()
        
    for j in range(epochs):
        
        
        accuracy=[]
                      
        loss=[]
        
        for i in range(training_iteration):
            
            senti_x,senti_y=load_Sentiment_Training_Batch()            
            all_tweets=np.concatenate((senti_x,Tweets_train[i]),axis=0)
            
            feed_dict={typhoon_X:training_features[i],typhoon_ytrue:training_labels[i],
                       tweets_inputs:all_tweets,tweets_labels:senti_y,tweets_Sizes:tweetsCounts_train[i]}

            sess.run([rnn_opt],feed_dict=feed_dict)
            
            loss_value,accuracy_value=sess.run([joint_Loss,joint_accuracy],feed_dict=feed_dict)
                      
            
            accuracy.append(accuracy_value)
            loss.append(loss_value)
                                
        value = tf.Summary.Value(tag="Loss",simple_value=np.mean(loss))
        summary = tf.Summary(value=[value])
        writer1.add_summary(summary,j)
          
                                                                               
        value = tf.Summary.Value(tag="Accuracy",simple_value=np.mean(accuracy))
        summary = tf.Summary(value=[value])
        writer1.add_summary(summary,j)

        # Evaluating testdata
        accuracy=[]
        loss=[]
        precision=[]
        recall=[]
        f1=[]
            
        for i in range(testing_iteration):
                    
          senti_x,senti_y=load_Sentiment_Testing_Batch()
            
          all_tweets=np.concatenate((senti_x,Tweets_test[i]),axis=0)
          
          feed_dict={typhoon_X:testing_features[i],typhoon_ytrue:testing_labels[i],
                   tweets_inputs:all_tweets,tweets_labels:senti_y,tweets_Sizes:tweetsCounts_test[i]}
          
          loss_value,accuracy_value,typhoon_ypred_value=sess.run([joint_Loss,joint_accuracy,y_p],feed_dict=feed_dict)
          
          accuracy.append(accuracy_value)
          loss.append(loss_value)
          
     
          y_true = np.argmax(testing_labels[i],1)
          
          precision.append(precision_score(y_true, typhoon_ypred_value,average='weighted'))
          recall.append(recall_score(y_true, typhoon_ypred_value,average='weighted'))
          f1.append(f1_score(y_true, typhoon_ypred_value,average='weighted'))

          
        value = tf.Summary.Value(tag="Loss",simple_value=np.mean(loss))
        summary = tf.Summary(value=[value])
        writer2.add_summary(summary,j)
                    
        value = tf.Summary.Value(tag="Accuracy",simple_value=np.mean(accuracy))
        summary2 = tf.Summary(value=[value])
        writer2.add_summary(summary2,j)
        
        
    print ("Performance Results (Accuracy, Precision, Recall and F1-Score):",np.mean(accuracy),np.mean(precision),np.mean(recall),np.mean(f1))                     
    
    writer1.close() 
    writer2.close()     